# Simulated PIK statistics

Here we inspect the accuracy and characteristics of the PIKs assigned,
leveraging our knowledge of ground truth from pseudopeople.

It wouldn't be possible to do the ground truth part with the real PVS, but
Layne, Wagner, and Rothhaas did something similar by redacting SSN from real records,
sending them through PVS without the SSN, and then using the true SSN
as ground truth.
The health care records they used are probably quite different from a CUF,
but they found a **very** good overall PIK accuracy (see cell below).

In [1]:
# Query planning is now on by default, but it has some rough edges.
# See https://github.com/dask/dask/issues/10995 for general discussion
# and https://github.com/dask/dask-expr/issues/1060 for the particular
# issue I ran into.
import dask
dask.config.set({"dataframe.query-planning": False})

In [2]:
import datetime, os, time

from vivarium_research_prl import distributed_compute, utils
from IPython.display import display

In [3]:
print(datetime.datetime.now())

2024-05-28 10:04:22.288677


In [4]:
# DO NOT EDIT if this notebook is not called ground_truth_accuracy.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
data_to_use = 'small_sample'
simulated_data_output_dir = 'output/generate_simulated_data'
case_study_output_dir = 'output'

# The "compute engine" is what we use on the Python side
# for our case-study-specific operations,
# as opposed to the Splink engine
compute_engine = 'pandas'
# Only matter if using a distributed compute engine
compute_engine_num_jobs = 3
compute_engine_cpus_per_job = 2
compute_engine_memory_per_job = "5GB"
queue = "long.q"
local_directory = f"/tmp/{os.environ['USER']}_{int(time.time())}_dask"

In [5]:
# Parameters
data_to_use = "small_sample"
simulated_data_output_dir = "output/generate_simulated_data/"
case_study_output_dir = "output/results/"
compute_engine = "pandas"


In [6]:
if compute_engine.startswith('dask'):
    utils.ensure_empty(local_directory)

In [7]:
case_study_output_dir = f'{case_study_output_dir}/{data_to_use}'
simulated_data_output_dir = f'{simulated_data_output_dir}/{data_to_use}'

In [8]:
df_ops, pd = distributed_compute.start_compute_engine(
    compute_engine,
    num_jobs=compute_engine_num_jobs,
    cpus_per_job=compute_engine_cpus_per_job,
    memory_per_job=compute_engine_memory_per_job,
    queue=queue,
    local_directory=local_directory,
)

In [9]:
census_2030_piked = df_ops.read_parquet(f'{case_study_output_dir}/census_2030_piked.parquet')
confirmed_piks_with_ground_truth = df_ops.read_parquet(f'{case_study_output_dir}/confirmed_piks.parquet')

In [10]:
piked_proportion = df_ops.compute(census_2030_piked.pik.notnull().mean())
# Compare with 90.28% of input records PIKed in the 2010 CUF,
# as reported in Wagner and Layne, Table 2, p. 18 
print(f'{piked_proportion:.2%} of the input records were PIKed')

89.38% of the input records were PIKed


In [11]:
# Multiple Census rows assigned the same PIK, indicating the model thinks they are duplicates in Census
pik_sizes = df_ops.persist(df_ops.groupby_agg_small_groups(census_2030_piked, by='pik', agg_func=lambda x: x.size()))
df_ops.compute(pik_sizes.value_counts())

1    9772
2      49
Name: count, dtype: int64

In [12]:
# Interesting: in pseudopeople, sometimes siblings are assigned the same (common) first name, making them almost identical.
# The only giveaway is their age and DOB.
# Presumably, this tends not to happen in real life.
duplicate_piks = pik_sizes.rename('pik_size').reset_index().pipe(lambda df: df[df.pik_size > 1])

df_ops.head(census_2030_piked.merge(duplicate_piks, on="pik").sort_values('pik'))

,record_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year,pik,pik_size
66,simulated_census_2030_8918,0_3036,Luther,M,Lovell,59,05/19/2003,38,glenmont rd nw,None,Anytown,WA,00000,Household,Reference person,Male,White,2030,100478,2
67,simulated_census_2030_8919,0_3036,Amarie,R,Lovell,26,05/19/2003,38,glenmont rd nw,None,Anytown,WA,00000,Household,Biological child,Male,White,2030,100478,2
62,simulated_census_2030_8875,0_11363,Aniyah,C,Mcanally,25,02/11/2005,n3140,silver eagle cir,None,Anytown,WA,00000,Household,Reference person,Female,White,2030,100935,2
63,simulated_census_2030_8881,0_11363,Lana,A,Mcanally,1,02/11/2005,n3140,silver eagle cir,None,Anytown,WA,00000,Household,Biological child,Female,White,2030,100935,2
45,simulated_census_2030_7621,0_7773,Kelly,C,Duane,60,02/17/2005,None,se 25th ave,None,Anytown,WA,00000,Household,Opposite-sex spouse,Female,White,2030,100937,2
46,simulated_census_2030_7622,0_7773,James,L,Duane,25,02/17/2005,None,se 25th ave,None,Anytown,WA,00000,Household,Biological child,Male,White,2030,100937,2
7,simulated_census_2030_2294,0_3,Valerie,M,Tsai,23,03/18/2006,5203,west farwell avenue,None,Anytown,WA,00000,College,Noninstitutionalized group quarters population,Female,Asian,2030,101216,2
97,simulated_census_2030_11042,0_7600,Valerie,M,Tsai,23,03/18/2006,7437,n cedar crest blvd,None,Anytown,WA,00000,Household,Other relative,Female,Asian,2030,101216,2
22,simulated_census_2030_4545,0_23,Elizabeth,O,Clegg,18,01/31/2012,5862,gordon lane,None,Anytown,WA,00000,Household,Biological child,Female,Black,2030,102596,2
21,simulated_census_2030_4544,0_23,Desiree,OP,Clegg,37,01/31/2012,5862,gordon lane,None,Anytown,WA,00000,Household,Reference person,Female,Black,2030,102596,2


## Ground truth statistics

In [13]:
census_2030_ground_truth = df_ops.persist(
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_census_2030_ground_truth.parquet')
)

In [14]:
# In this version of pseudopeople, there are no actual duplicates in Census,
# which means all of the duplicates identified above are wrong.
assert len(census_2030_ground_truth) == len(df_ops.drop_duplicates(census_2030_ground_truth))

In [15]:
reference_files_ground_truth = df_ops.persist(df_ops.concat([
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_geobase_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
    df_ops.read_parquet(f'{simulated_data_output_dir}/simulated_name_dob_reference_file_ground_truth.parquet').drop(columns=['n_unique_simulants']),
], ignore_index=True))

In [16]:
# However, there can be reference file records that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(reference_files_ground_truth, by='record_id', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1    51726
2     1294
3       41
Name: count, dtype: int64

In [17]:
reference_files_ground_truth = df_ops.persist(reference_files_ground_truth.merge(
    n_unique_simulants,
    on='record_id',
    how='left',
))
reference_files_ground_truth.head(n=100)

,record_id,simulant_id,n_unique_simulants
0,simulated_geobase_reference_file_26802,0_1366,1
1,simulated_geobase_reference_file_39,0_5212,1
2,simulated_geobase_reference_file_27360,0_5595,1
3,simulated_geobase_reference_file_108,0_14382,1
4,simulated_geobase_reference_file_109,0_15523,1
...,...,...,...
95,simulated_geobase_reference_file_1015,0_6758,1
96,simulated_geobase_reference_file_1028,0_8466,1
97,simulated_geobase_reference_file_1037,0_11611,1
98,simulated_geobase_reference_file_1042,0_6520,1


In [18]:
df_ops.head(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == df_ops.compute(reference_files_ground_truth.n_unique_simulants.max())])

,record_id,simulant_id,n_unique_simulants
5657,simulated_geobase_reference_file_7286,0_6717,3
6004,simulated_geobase_reference_file_23158,0_9933,3
6123,simulated_geobase_reference_file_30441,0_12219,3
6124,simulated_geobase_reference_file_30444,0_12219,3
6225,simulated_geobase_reference_file_23368,0_10345,3
7503,simulated_geobase_reference_file_29432,0_7701,3
8607,simulated_geobase_reference_file_20222,0_14938,3
8684,simulated_geobase_reference_file_17640,0_20175,3
8857,simulated_geobase_reference_file_16008,0_6503,3
8919,simulated_geobase_reference_file_28463,0_21744,3


In [19]:
census_2030_ground_truth = df_ops.persist(census_2030_ground_truth.merge(
    df_ops.drop_duplicates(reference_files_ground_truth[['simulant_id']]).assign(possible_to_pik=1),
    on='simulant_id',
    how='left',
).assign(possible_to_pik=lambda df: df.possible_to_pik.fillna(0)))
possible_to_pik_proportion = df_ops.compute(census_2030_ground_truth.possible_to_pik.mean())
print(
    f'{(1 - possible_to_pik_proportion):.2%} of the input records are '
    'impossible to PIK correctly, since they are not in any reference files'
)

0.45% of the input records are impossible to PIK correctly, since they are not in any reference files


In [20]:
print(
    f'Assigned PIKs to {(piked_proportion / possible_to_pik_proportion):.2%} of PIK-able records'
)

Assigned PIKs to 89.78% of PIK-able records


In [21]:
reference_file = df_ops.concat([
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_geobase_reference_file.parquet',
    ),
    df_ops.read_parquet(
        f'{simulated_data_output_dir}/simulated_name_dob_reference_file.parquet',
    ),
], ignore_index=True)

In [22]:
reference_file_piks = df_ops.persist(reference_file[['record_id', 'pik']])
reference_file_piks

,record_id,pik
0,simulated_geobase_reference_file_0,106067
1,simulated_geobase_reference_file_1,104228
2,simulated_geobase_reference_file_2,106714
3,simulated_geobase_reference_file_3,105169
4,simulated_geobase_reference_file_4,106386
...,...,...
53056,simulated_name_dob_reference_file_20105,109193
53057,simulated_name_dob_reference_file_20106,109194
53058,simulated_name_dob_reference_file_20107,109195
53059,simulated_name_dob_reference_file_20108,109196


In [23]:
assert len(reference_file_piks) == len(df_ops.drop_duplicates(reference_file_piks[['record_id']]))

In [24]:
pik_simulant_pairs = df_ops.persist(df_ops.drop_duplicates(reference_files_ground_truth.merge(reference_file_piks, on='record_id')[['pik', 'simulant_id']]))

In [25]:
# However, there can be PIKs that correspond to multiple simulants,
# due to errors in the reference file construction by SSN
n_unique_simulants = df_ops.persist(df_ops.groupby_agg_small_groups(pik_simulant_pairs, by='pik', agg_func=lambda x: x.simulant_id.nunique()).rename('n_unique_simulants').reset_index())
df_ops.compute(n_unique_simulants.n_unique_simulants.value_counts())

n_unique_simulants
1    17885
2     1042
3       51
Name: count, dtype: int64

In [26]:
pik_simulant_pairs = df_ops.persist(pik_simulant_pairs.merge(
    n_unique_simulants,
    on='pik',
    how='left',
))
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
0,106715,0_1985,2
1,92228,0_12419,2
2,98332,0_4901,2
3,104603,0_17547,2
4,105384,0_16443,2
...,...,...,...
20117,109193,0_22674,1
20118,109194,0_21507,1
20119,109195,0_7328,1
20120,109196,0_22299,1


In [27]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == df_ops.compute(pik_simulant_pairs.n_unique_simulants.max())])

,pik,simulant_id,n_unique_simulants
15,95498,0_21546,3
30,93259,0_6717,3
54,107422,0_12219,3
62,102188,0_10345,3
66,89608,0_13319,3
122,105812,0_22559,3
129,106724,0_7701,3
148,105434,0_6536,3
196,99125,0_20175,3
208,98203,0_6503,3


## Definitions of accuracy

1. (most strict) Assigning any PIK with multiple simulants is incorrect
2. Assigning a PIK with multiple simulants is neither incorrect nor correct (excluded from denominator)
3. (most lenient) Assigning a PIK with multiple simulants is correct, as long as at least one of those simulants matches the truth

In [28]:
# All modules, Medicare database, calculated from Layne, Wagner, and Rothhaas Table 1 (p. 15)
real_life_pvs_accuracy = 1 - (2_585 + 60_709 + 129_480 + 89_094) / (52_406_981 + 5_170_924 + 49_374_794 + 50_327_034)
f'{real_life_pvs_accuracy:.5%}'

'99.82079%'

### Definition 1

In [29]:
piks_assigned = df_ops.compute(census_2030_piked.pik.notnull().sum())
piks_assigned

9870

In [30]:
df_ops.head(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants > 1])

,pik,simulant_id,n_unique_simulants
0,106715,0_1985,2
1,92228,0_12419,2
2,98332,0_4901,2
3,104603,0_17547,2
4,105384,0_16443,2
5,103385,0_10377,2
6,105187,0_3392,2
7,93465,0_15123,2
8,105480,0_11997,2
9,105101,0_5102,2


In [31]:
single_sim_piks_correct = df_ops.compute(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_piks_correct

9015

In [32]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / piks_assigned
)

0.9133738601823708

In [33]:
assert len(confirmed_piks_with_ground_truth) == piks_assigned

In [34]:
df_ops.head(census_2030_ground_truth.rename(columns={'record_id': 'record_id_census_2030'}))

,record_id_census_2030,simulant_id,possible_to_pik
0,simulated_census_2030_0,0_923,1.0
1,simulated_census_2030_1,0_2348,1.0
2,simulated_census_2030_2,0_2641,1.0
3,simulated_census_2030_3,0_6176,1.0
4,simulated_census_2030_4,0_10251,1.0
5,simulated_census_2030_5,0_13047,1.0
6,simulated_census_2030_6,0_13861,1.0
7,simulated_census_2030_7,0_13972,1.0
8,simulated_census_2030_8,0_13973,1.0
9,simulated_census_2030_9,0_13974,1.0


In [35]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_correct = df_ops.compute(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .pipe(lambda df: (df.simulant_id_x == df.simulant_id_y) & (df.n_unique_simulants == 1))
        .sum()
)
single_sim_record_links_correct

9216

In [36]:
(
    single_sim_record_links_correct / piks_assigned
)

0.9337386018237082

### Definition 2

In [37]:
single_sim_piks_assigned = len(census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs[pik_simulant_pairs.n_unique_simulants == 1][['pik', 'simulant_id']]))
single_sim_piks_assigned

9052

In [38]:
# Overall accuracy, treating it as a black box
(
    single_sim_piks_correct / single_sim_piks_assigned
)

0.9959125055236412

In [39]:
# Looking at whether the exact *record* linked was from the same simulant
single_sim_record_links_assigned = df_ops.compute(
    (confirmed_piks_with_ground_truth
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .n_unique_simulants == 1).sum()
)
single_sim_record_links_assigned

9253

In [40]:
(
    single_sim_record_links_correct / single_sim_record_links_assigned
)

0.9960012968766886

### Definition 3

In [41]:
pik_simulant_pairs

,pik,simulant_id,n_unique_simulants
0,106715,0_1985,2
1,92228,0_12419,2
2,98332,0_4901,2
3,104603,0_17547,2
4,105384,0_16443,2
...,...,...,...
20117,109193,0_22674,1
20118,109194,0_21507,1
20119,109195,0_7328,1
20120,109196,0_22299,1


In [42]:
piks_at_least_partially_correct = df_ops.persist(
    census_2030_piked[['record_id', 'pik']].merge(pik_simulant_pairs, on='pik').merge(census_2030_ground_truth, on='record_id')
        .pipe(df_ops.drop_duplicates)
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id", "pik"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
piks_at_least_partially_correct

,record_id,pik,correct
0,simulated_census_2030_0,89638,True
1,simulated_census_2030_1,98895,True
2,simulated_census_2030_10,94642,True
3,simulated_census_2030_100,100994,True
4,simulated_census_2030_1000,93336,True
...,...,...,...
9865,simulated_census_2030_9995,101140,True
9866,simulated_census_2030_9996,95441,True
9867,simulated_census_2030_9997,101716,True
9868,simulated_census_2030_9998,98692,True


In [43]:
# Overall accuracy, treating it as a black box
piks_correct_proportion = (df_ops.compute(piks_at_least_partially_correct.correct.sum()) / piks_assigned)
piks_correct_proportion

0.996048632218845

In [44]:
print(f'{piks_correct_proportion:.5%} of the PIKs assigned were correct; compare with {real_life_pvs_accuracy:.5%} in real life')

99.60486% of the PIKs assigned were correct; compare with 99.82079% in real life


In [45]:
# Looking at whether the exact *record* linked was from the same simulant
sim_record_links_at_least_partially_correct = df_ops.persist(
    confirmed_piks_with_ground_truth
        .merge(
            census_2030_ground_truth.rename(columns={'record_id': 'record_id_raw_input_file'}),
            on='record_id_raw_input_file',
        )
        .merge(
            reference_files_ground_truth.rename(columns={'record_id': 'record_id_reference_file'}),
            on='record_id_reference_file',
        )
        .assign(correct=lambda df: df.simulant_id_x == df.simulant_id_y)
        .pipe(df_ops.groupby_agg_small_groups, by=["record_id_raw_input_file", "record_id_reference_file", "pik", "module_name", "pass_name"], agg_func=lambda x: x.correct.any())
        .reset_index()
)
sim_record_links_at_least_partially_correct

,record_id_raw_input_file,record_id_reference_file,pik,module_name,pass_name,correct
0,simulated_census_2030_0,simulated_geobase_reference_file_959,89638,geosearch,geokey,True
1,simulated_census_2030_1,simulated_geobase_reference_file_17209,98895,geosearch,geokey,True
2,simulated_census_2030_10,simulated_geobase_reference_file_9701,94642,geosearch,geokey,True
3,simulated_census_2030_100,simulated_geobase_reference_file_21104,100994,geosearch,geokey,True
4,simulated_census_2030_1000,simulated_geobase_reference_file_7422,93336,geosearch,some name and DOB information,True
...,...,...,...,...,...,...
9865,simulated_census_2030_9995,simulated_geobase_reference_file_21380,101140,hhcompsearch,year of birth,True
9866,simulated_census_2030_9996,simulated_geobase_reference_file_11091,95441,geosearch,geokey,True
9867,simulated_census_2030_9997,simulated_geobase_reference_file_22501,101716,geosearch,geokey,True
9868,simulated_census_2030_9998,simulated_geobase_reference_file_16873,98692,geosearch,house number and street name Soundex,True


In [46]:
len(sim_record_links_at_least_partially_correct)

9870

In [47]:
len(df_ops.drop_duplicates(sim_record_links_at_least_partially_correct[['record_id_raw_input_file', 'record_id_reference_file']]))

9870

In [48]:
(
    df_ops.compute(sim_record_links_at_least_partially_correct.correct.sum()) / piks_assigned
)

0.996048632218845

In [49]:
assert df_ops.compute((df_ops.groupby_agg_small_groups(confirmed_piks_with_ground_truth, by='record_id_raw_input_file', agg_func=lambda x: x.record_id_reference_file.nunique()) <= 1).all())

In [50]:
# Using definition 3 -- at the PIK level
piks_at_least_partially_correct = df_ops.persist(
    piks_at_least_partially_correct
        .rename(columns={'record_id': 'record_id_raw_input_file'})
        .merge(confirmed_piks_with_ground_truth[['record_id_raw_input_file', 'module_name', 'pass_name']], on='record_id_raw_input_file')
)
piks_at_least_partially_correct

,record_id_raw_input_file,pik,correct,module_name,pass_name
0,simulated_census_2030_0,89638,True,geosearch,geokey
1,simulated_census_2030_1,98895,True,geosearch,geokey
2,simulated_census_2030_10,94642,True,geosearch,geokey
3,simulated_census_2030_100,100994,True,geosearch,geokey
4,simulated_census_2030_1000,93336,True,geosearch,some name and DOB information
...,...,...,...,...,...
9865,simulated_census_2030_9995,101140,True,hhcompsearch,year of birth
9866,simulated_census_2030_9996,95441,True,geosearch,geokey
9867,simulated_census_2030_9997,101716,True,geosearch,geokey
9868,simulated_census_2030_9998,98692,True,geosearch,house number and street name Soundex


In [51]:
# Accuracy by module -- note that this shows the opposite pattern (with the sample data)
# relative to the results of Layne et al., who found GeoSearch was much *more* accurate
df_ops.compute(piks_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
geosearch,0.995796,9277
dobsearch,1.000000,166
hhcompsearch,1.000000,39
namesearch,1.000000,388


In [52]:
# Accuracy by pass -- could be used to tune pass-specific cutoffs, but
# this might not be too informative while we are still using the sample data.
df_ops.compute(piks_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                      
geosearch    geokey                                              0.994335   
             house number and street name Soundex                0.998258   
dobsearch    initials name switch                                1.000000   
             first two characters of first name and year of ...  1.000000   
             reverse Soundex of name                             1.000000   
geosearch    geokey name switch                                  1.000000   
             house number and street name Soundex name switch    1.000000   
             some name and DOB information                       1.000000   
hhcompsearch initials                                            1.000000   
             year of birth                                       1.000000   
namesearch   DOB and NYSIIS of name                              1.000000   
             DOB and initials                                    1.000000   
             birthday and first two characters of name           1.000000   
             year of birth and first two characters of name      1.000000   

                                                                 size  
module_name  pass_name                                                 
geosearch    geokey                                              6708  
             house number and street name Soundex                 574  
dobsearch    initials name switch                                   5  
             first two characters of first name and year of ...   120  
             reverse Soundex of name                               41  
geosearch    geokey name switch                                    45  
             house number and street name Soundex name switch       2  
             some name and DOB information                       1948  
hhcompsearch initials                                              31  
             year of birth                                          8  
namesearch   DOB and NYSIIS of name                               219  
             DOB and initials                                      97  
             birthday and first two characters of name             61  
             year of birth and first two characters of name        11

In [53]:
# Using definition 3 -- at the link level
df_ops.compute(sim_record_links_at_least_partially_correct.groupby("module_name").correct.agg(["mean", "size"]).sort_values("mean"))

,mean,size
module_name,,
geosearch,0.995796,9277
dobsearch,1.000000,166
hhcompsearch,1.000000,39
namesearch,1.000000,388


In [54]:
df_ops.compute(sim_record_links_at_least_partially_correct.groupby(["module_name", "pass_name"]).correct.agg(["mean", "size"]).sort_values("mean"))

mean  \
module_name  pass_name                                                      
geosearch    geokey                                              0.994335   
             house number and street name Soundex                0.998258   
dobsearch    initials name switch                                1.000000   
             first two characters of first name and year of ...  1.000000   
             reverse Soundex of name                             1.000000   
geosearch    geokey name switch                                  1.000000   
             house number and street name Soundex name switch    1.000000   
             some name and DOB information                       1.000000   
hhcompsearch initials                                            1.000000   
             year of birth                                       1.000000   
namesearch   DOB and NYSIIS of name                              1.000000   
             DOB and initials                                    1.000000   
             birthday and first two characters of name           1.000000   
             year of birth and first two characters of name      1.000000   

                                                                 size  
module_name  pass_name                                                 
geosearch    geokey                                              6708  
             house number and street name Soundex                 574  
dobsearch    initials name switch                                   5  
             first two characters of first name and year of ...   120  
             reverse Soundex of name                               41  
geosearch    geokey name switch                                    45  
             house number and street name Soundex name switch       2  
             some name and DOB information                       1948  
hhcompsearch initials                                              31  
             year of birth                                          8  
namesearch   DOB and NYSIIS of name                               219  
             DOB and initials                                      97  
             birthday and first two characters of name             61  
             year of birth and first two characters of name        11

In [55]:
df_ops.compute(sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct].groupby(["module_name", "pass_name"]).size()).sort_values()

module_name  pass_name                           
geosearch    house number and street name Soundex     1
             geokey                                  38
dtype: int64

### Incorrect and missed PIKs

In [56]:
incorrectly_linked_pairs = df_ops.persist(df_ops.drop_duplicates(
    sim_record_links_at_least_partially_correct[~sim_record_links_at_least_partially_correct.correct]
        [["record_id_raw_input_file", "record_id_reference_file"]]
))
incorrectly_linked_pairs

,record_id_raw_input_file,record_id_reference_file
275,simulated_census_2030_10272,simulated_geobase_reference_file_29147
386,simulated_census_2030_1038,simulated_geobase_reference_file_12100
502,simulated_census_2030_10493,simulated_geobase_reference_file_9885
726,simulated_census_2030_10713,simulated_geobase_reference_file_15669
831,simulated_census_2030_10816,simulated_geobase_reference_file_30309
1789,simulated_census_2030_187,simulated_geobase_reference_file_14326
1931,simulated_census_2030_2000,simulated_geobase_reference_file_6561
2353,simulated_census_2030_2468,simulated_geobase_reference_file_26002
2391,simulated_census_2030_2503,simulated_geobase_reference_file_9144
3625,simulated_census_2030_3780,simulated_geobase_reference_file_7894


In [57]:
len(incorrectly_linked_pairs)

39

In [58]:
incorrect_links = df_ops.head(incorrectly_linked_pairs, n=100)
incorrect_links

,record_id_raw_input_file,record_id_reference_file
275,simulated_census_2030_10272,simulated_geobase_reference_file_29147
386,simulated_census_2030_1038,simulated_geobase_reference_file_12100
502,simulated_census_2030_10493,simulated_geobase_reference_file_9885
726,simulated_census_2030_10713,simulated_geobase_reference_file_15669
831,simulated_census_2030_10816,simulated_geobase_reference_file_30309
1789,simulated_census_2030_187,simulated_geobase_reference_file_14326
1931,simulated_census_2030_2000,simulated_geobase_reference_file_6561
2353,simulated_census_2030_2468,simulated_geobase_reference_file_26002
2391,simulated_census_2030_2503,simulated_geobase_reference_file_9144
3625,simulated_census_2030_3780,simulated_geobase_reference_file_7894


In [59]:
%xdel incorrectly_linked_pairs

In [60]:
comparison_cols = [
    "first_name",
    "middle_name",
    "last_name",
    "date_of_birth",
    "street_number",
    "street_name",
    "unit_number",
    "city",
    "state",
]

incorrect_links_detail = (
    incorrect_links
        .merge(
            df_ops.compute(census_2030_piked[census_2030_piked.record_id.isin(incorrect_links.record_id_raw_input_file)])
                .rename(columns={"record_id": "record_id_raw_input_file", "middle_initial": "middle_name"})
                [["record_id_raw_input_file"] + comparison_cols],
            on="record_id_raw_input_file",
            how="left",
        )
        .merge(
            df_ops.compute(reference_file[reference_file.record_id.isin(incorrect_links.record_id_reference_file)])
                .rename(columns={"record_id": "record_id_reference_file"})
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                [["record_id_reference_file"] + comparison_cols],
            on="record_id_reference_file",
            how="left",
            suffixes=("_census", "_reference_file"),
        )
)
def flatten(xss):
    return [x for xs in xss for x in xs]

incorrect_links_detail[flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])]

,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,Charlene,Mayra,L,Alaya,Stockman,Stockman,05/22/2025,20250522,13711,13711,filbert st,FILBERT ST,None,None,Anytown,ANYTOWN,WA,WA
1,Aniylah,Kristin,R,Michelle,Reeder,Reeder,12/29/1982,19821229,6302,6302,n carleton ave,N CARLETON AVE,None,None,Anytown,ANYTOWN,WA,WA
2,Elroy,Cecflia,J,Karen,Abraham,Abeaham,08/17/1976,19760817,1242,1242,ashburton rd,ASHBURTON RD,None,None,Anytown,ANYTOWN,WA,WA
3,Jossph,Dylan,B,Jimmy,Hartwell,Hartwell,07/15/1992,19920715,225,225,av 360,AV 360,None,None,Anytown,ANYTOWN,WA,WA
4,Amara,King,L,Louis,Vanover,Vanover,06/08/2027,20270608,None,None,sw 178th st,SW 178TH ST,None,None,Anytown,ANYTOWN,WA,WA
5,Remington,Domonique,F,Jacquelyn,Woods,Woods,12/11/1988,19881211,8173,8173,meridian ave nth,MERIDIAN AVE NTH,None,None,Anytown,ANYTOWN,WA,WA
6,Carolyn,Mark,E,Daniel,Tamariz,Tamariz,07/12/1966,19660712,3809,3809,brier creek ct,BRIER CREEK CT,None,None,Anytown,ANYTOWN,WA,WA
7,Vivian,Tilly,M,Spencer,Ervin,Ervin,07/01/2019,20190701,11711,11711,sherwood st,SHERWOOD ST,None,None,Anytown,ANYTOWN,WA,WA
8,Jorge,Colleen,C,Carol,Patel-Gallardo,Patel-Gallardo,07/18/1974,19740718,1850,1850,via blairo,VIA BLAIRO,None,None,Anytown,ANYTOWN,WA,WA
9,Angel,Darryl,J,Gregory,Mata-Gonzalez,Mata-Gonzalez,08/26/1970,19700826,11990,11990,n 18th st,N 18TH ST,None,None,Anytown,ANYTOWN,WA,WA


In [61]:
missed_links = df_ops.persist(
    census_2030_piked[census_2030_piked.pik.isnull()][["record_id"]]
        .merge(census_2030_ground_truth, on="record_id")
        .merge(reference_files_ground_truth[reference_files_ground_truth.n_unique_simulants == 1], on="simulant_id", suffixes=("_census", "_reference_file"))
)

In [62]:
len(missed_links)

3290

In [63]:
simulants_missed = df_ops.head(missed_links[['simulant_id']], n=100).simulant_id.unique()
simulants_missed

array(['0_10251', '0_13861', '0_9', '0_21615', '0_21616', '0_21',
       '0_2154', '0_11857', '0_18062', '0_17384', '0_5474', '0_18927',
       '0_12284', '0_4702', '0_21693', '0_22602', '0_4991', '0_7310',
       '0_12456', '0_17692', '0_4445', '0_2048', '0_2538', '0_74',
       '0_1471', '0_1611', '0_1637', '0_1980', '0_2057', '0_2144',
       '0_2767', '0_2957', '0_2990', '0_3536', '0_4519', '0_4583'],
      dtype=object)

In [64]:
missed_pairs = df_ops.compute(missed_links[missed_links.simulant_id.isin(list(simulants_missed))])
missed_pairs

,record_id_census,simulant_id,possible_to_pik,record_id_reference_file,n_unique_simulants
0,simulated_census_2030_4,0_10251,1.0,simulated_geobase_reference_file_13057,1
1,simulated_census_2030_4,0_10251,1.0,simulated_geobase_reference_file_13058,1
2,simulated_census_2030_4,0_10251,1.0,simulated_geobase_reference_file_13059,1
3,simulated_census_2030_4,0_10251,1.0,simulated_name_dob_reference_file_7451,1
4,simulated_census_2030_6,0_13861,1.0,simulated_geobase_reference_file_11013,1
...,...,...,...,...,...
95,simulated_census_2030_306,0_4519,1.0,simulated_geobase_reference_file_23629,1
96,simulated_census_2030_306,0_4519,1.0,simulated_geobase_reference_file_23630,1
97,simulated_census_2030_306,0_4519,1.0,simulated_name_dob_reference_file_13263,1
98,simulated_census_2030_308,0_4583,1.0,simulated_geobase_reference_file_16764,1


In [65]:
%xdel missed_links

In [66]:
missed_links_detail = (
    missed_pairs
        .merge(
            df_ops.compute(census_2030_piked[census_2030_piked.record_id.isin(list(missed_pairs.record_id_census))])
                .rename(columns={"record_id": "record_id_census", "middle_initial": "middle_name"}),
            on="record_id_census",
        )
        .merge(
            df_ops.compute(reference_file[reference_file.record_id.isin(missed_pairs.record_id_reference_file)])
                .rename(columns=lambda c: c.replace('mailing_address_', ''))
                .rename(columns={"record_id": "record_id_reference_file"}),
            on="record_id_reference_file",
            suffixes=("_census", "_reference_file"),
        )
)

In [67]:
for simulant in simulants_missed:
    print(simulant)
    display(missed_links_detail[missed_links_detail.simulant_id == simulant][['simulant_id'] + flatten([(f'{c}_census', f'{c}_reference_file') for c in comparison_cols])])

0_10251


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
0,0_10251,Bobby,Bpbbu,S,Shane,Rhimpson,Thompson,05/30/1985,19850520,None,None,winding trail rd,WINDING TRAIL RD,None,None,Anytown,ANYTOWN,WA,WA
1,0_10251,Bobby,Bpbbu,S,Shane,Rhimpson,Thompson,05/30/1985,19850520,None,17861,winding trail rd,WINDING TRAIL RD,None,None,Anytown,ANYTOWN,WA,WA
2,0_10251,Bobby,Bpbbu,S,Shane,Rhimpson,Thompson,05/30/1985,19850520,None,17868,winding trail rd,WINDING TRAIL RD,None,None,Anytown,ANYTOWN,WA,WA
3,0_10251,Bobby,Bpbbu,S,Shane,Rhimpson,Thompson,05/30/1985,19850520,None,NaN,winding trail rd,NaN,None,NaN,Anytown,NaN,WA,NaN


0_13861


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
4,0_13861,Levi J,Levi,,Jared,Shirley,Sbirlev,01/19/198o,19800119,32597,32597,delacorte dr,DELACORTE DR,None,None,Anytown,ANYTOWN,WA,WA
5,0_13861,Levi J,Levi,,Jared,Shirley,Sbirlev,01/19/198o,19800119,32597,1307,delacorte dr,ROSEWOOD AVE,None,None,Anytown,ANYTOWN,WA,WA
6,0_13861,Levi J,Levi,,Jared,Shirley,Sbirlev,01/19/198o,19800119,32597,NaN,delacorte dr,NaN,None,NaN,Anytown,NaN,WA,NaN


0_9


...

,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
93,0_3536,Elyse,Elyse,M,Maria,Arceo,Arceo,31/06/2003,20030126,34,34,bowen cir sw,BOWEN CIR SW,None,None,Anytown,ANYTOWN,WA,WA
94,0_3536,Elyse,Elyse,M,Maria,Arceo,Arceo,31/06/2003,20030126,34,NaN,bowen cir sw,NaN,None,NaN,Anytown,NaN,WA,NaN


0_4519


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
95,0_4519,Gabriel,Wyatt,W,Gabriel,Simmons,Of House,12/20/2010,20101220,34,34,bowen cir sw,BOWEN CIR SW,None,None,Anytown,ANYTOWN,WA,WA
96,0_4519,Gabriel,Wyatt,W,Gabriel,Simmons,Of House,12/20/2010,20101220,34,37,bowen cir sw,BOWEN CIR SW,None,None,Anytown,AGYTOWN,WA,WA
97,0_4519,Gabriel,Wyatt,W,Gabriel,Simmons,Of House,12/20/2010,20101220,34,NaN,bowen cir sw,NaN,None,NaN,Anytown,NaN,WA,NaN


0_4583


,simulant_id,first_name_census,first_name_reference_file,middle_name_census,middle_name_reference_file,last_name_census,last_name_reference_file,date_of_birth_census,date_of_birth_reference_file,street_number_census,street_number_reference_file,street_name_census,street_name_reference_file,unit_number_census,unit_number_reference_file,city_census,city_reference_file,state_census,state_reference_file
98,0_4583,Garrett,None,W,Wyatt,Nickell,Nickell,07/29/1995,19950729,34,34,bowen cir sw,BOWEN CIR SW,None,None,Anydown,ANYTOWN,WA,WA
99,0_4583,Garrett,None,W,Wyatt,Nickell,Nickell,07/29/1995,19950729,34,NaN,bowen cir sw,NaN,None,NaN,Anydown,NaN,WA,NaN
